In [ ]:
pip uninstall scikit-learn transformers transformers torch PyMuPDF

In [ ]:
pip install scikit-learn

In [ ]:
pip install transformers

In [ ]:
pip install requests PyMuPDF

In [ ]:
pip install transformers torch

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import pipeline
import pandas as pd
import requests
import fitz 
import json
import os
import re

In [ ]:
def download_pdf(pdf_url, filename="downloaded.pdf"):
    response = requests.get(pdf_url)
    with open(filename, 'wb') as f:
        f.write(response.content)
    return filename

In [ ]:
def pdf_to_json(pdf_path, output_json="output.json"):
    doc = fitz.open(pdf_path)
    pdf_data = {}

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text = page.get_text()
        pdf_data[f"page_{page_num + 1}"] = text.strip()

    with open(output_json, 'w', encoding='utf-8') as json_file:
        json.dump(pdf_data, json_file, indent=2, ensure_ascii=False)

    for page, content in pdf_data.items():
        print(f"\n--- {page} ---\n{content}\n")

    return pdf_data

    # print(f"✅ PDF converted and saved to {output_json}")

In [ ]:
local_pdf = download_pdf("https://ifsca.gov.in/CommonDirect/GetFileView?id=21626bde60601ef44a0ed0220138e431&fileName=Regulatory_action_s__against_certain_Fund_Management_Entities_for_non_compliance_with_IFSCA__Fund_Management__Regulations__2025_20250718_0112.pdf")
pdf_to_json(local_pdf)

In [ ]:
print(local_pdf)

In [ ]:
with open("output.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [ ]:
def clean_text(text):
    text = re.sub(r"\[image\]", "", text)
    text = re.sub(r"[•■○]", "-", text)
    text = re.sub(r"-{2,}", "-", text)
    text = re.sub(r"[×]", "", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = re.sub(r"\n+", "\n", text)
    return text.strip()

In [ ]:
cleaned_data = {page: clean_text(content) for page, content in data.items()}

In [ ]:
with open("pdf_cleaned.json", "w", encoding="utf-8") as f:
    json.dump(cleaned_data, f, indent=2, ensure_ascii=False)

In [ ]:
with open("pdf_cleaned.json", "w", encoding="utf-8") as f:
    json.dump(cleaned_data, f, indent=2, ensure_ascii=False)

In [ ]:
with open("pdf_cleaned.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [ ]:
pages = [data[page] for page in sorted(data.keys(), key=lambda x: int(x.split('_')[1]))]

In [ ]:
vectorizer = TfidfVectorizer(
    stop_words='english',
    max_df=0.9, 
    min_df=1,
)

In [ ]:
tfidf_matrix = vectorizer.fit_transform(pages)

In [ ]:
df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())
df_tfidf.index = [f"page_{i+1}" for i in range(len(pages))]

In [ ]:
df_tfidf.to_csv("tfidf_matrix.csv", index=True)

In [ ]:
print(df_tfidf.head())

In [ ]:
summarizer = pipeline("summarization")


In [ ]:
with open("pdf_cleaned.json", "r", encoding="utf-8") as f:
    data = json.load(f)

In [ ]:
full_text = "\n".join(data[page] for page in sorted(data.keys(), key=lambda x: int(x.split('_')[1])))

In [ ]:
summary = summarizer(full_text, max_length=150, min_length=50, do_sample=False)

In [ ]:
print("Summary:\n", summary[0]['summary_text'])